In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import os

# 예시 데이터 (실제 데이터로 대체하세요)
df = pd.read_parquet('/workspace/infra-guidance/logs/A1_A1_test.parquet', engine='pyarrow')

# 시각화 함수 정의
def plot_trajectories(df, scenario_id):
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.set_xlim(-0.2, 0.3)  # X 축 범위
    ax.set_ylim(-0.15, 0.05)  # Y 축 범위
    ax.set_title(f'Predicted Trajectories for Scenario {scenario_id}')
    ax.set_xlabel('X coordinate')
    ax.set_ylabel('Y coordinate')

    lines = []
    for _ in range(len(df)):
        line, = ax.plot([], [], lw=2)
        lines.append(line)

    def update(frame):
        for i, line in enumerate(lines):
            x_data = df['predicted_trajectory_x'].iloc[i][:frame]
            y_data = df['predicted_trajectory_y'].iloc[i][:frame]
            line.set_data(x_data, y_data)
        return lines

    anim = FuncAnimation(fig, update, frames=len(df['predicted_trajectory_x'].iloc[0]), interval=500, blit=True)
    
    # 결과를 저장할 폴더가 없으면 생성
    if not os.path.exists('videos'):
        os.makedirs('videos')

    # 각 시나리오에 대한 동영상 파일 저장
    anim.save('/workspace/infra-guidance/logs/{}_{}_trajectory_animation.gif'.format('A1_A1_test', scenario_id), writer='imagemagick')

    plt.close(fig)  # 애니메이션 후 그래프를 닫기

# 시나리오 별로 필터링하여 개별 동영상 생성
for scenario_id in df['scenario_id'].unique():
    scenario_df = df[df['scenario_id'] == scenario_id]
    plot_trajectories(scenario_df, scenario_id)


In [ ]:
df = pd.read_parquet('/noah/dataset/infra-guidance/SUMO/sumo_process/A1/test/raw/1_1000000/scenario_1_1000000.parquet', engine='pyarrow')
df.head()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import os

df = pd.read_parquet('/workspace/infra-guidance/logs/A1_A1_test.parquet', engine='pyarrow')
max_scene = 10

# 시각화 함수 정의
def plot_trajectories(df, scenario_id):
    fig, ax = plt.subplots(figsize=(10, 6))
    
    # X, Y 축의 최소/최대값을 계산하여 축 범위 설정
    x_min = min([min(x) for x in df['predicted_trajectory_x']])
    x_max = max([max(x) for x in df['predicted_trajectory_x']])
    y_min = min([min(y) for y in df['predicted_trajectory_y']])
    y_max = max([max(y) for y in df['predicted_trajectory_y']])
    
    ax.set_xlim(x_min, x_max)  # X 축 범위 동적 설정
    ax.set_ylim(y_min, y_max)  # Y 축 범위 동적 설정
    
    ax.set_title(f'Predicted Trajectories for Scenario {scenario_id}')
    ax.set_xlabel('X coordinate')
    ax.set_ylabel('Y coordinate')

    lines = []
    for _ in range(len(df)):
        line, = ax.plot([], [], lw=2)
        lines.append(line)

    def update(frame):
        for i, line in enumerate(lines):
            x_data = df['predicted_trajectory_x'].iloc[i][:frame]
            y_data = df['predicted_trajectory_y'].iloc[i][:frame]
            line.set_data(x_data, y_data)
        return lines

    anim = FuncAnimation(fig, update, frames=len(df['predicted_trajectory_x'].iloc[0]), interval=500, blit=True)
    
    # 결과를 저장할 폴더가 없으면 생성
    if not os.path.exists('videos'):
        os.makedirs('videos')

    # 각 시나리오에 대한 동영상 파일 저장
    anim.save('/workspace/infra-guidance/logs/{}_{}_trajectory_animation.gif'.format('A1_A1_test', scenario_id), writer='imagemagick')

    plt.close(fig)  # 애니메이션 후 그래프를 닫기

# 시나리오 별로 필터링하여 개별 동영상 생성
for idx, scenario_id in enumerate(df['scenario_id'].unique()):
    if idx==max_scene:
        break
    scenario_df = df[df['scenario_id'] == scenario_id]
    plot_trajectories(scenario_df, scenario_id)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import os

# GT 및 예측 데이터를 DataFrame으로 변환
scenario_id = '1_1000360'

gt_df = pd.read_parquet('/noah/dataset/infra-guidance/SUMO/sumo_process/A1/test/raw/{}/scenario_{}.parquet'.format(scenario_id,scenario_id), engine='pyarrow')
predicted_df = pd.read_parquet('/workspace/infra-guidance/logs/A1_A1_test.parquet', engine='pyarrow')
predicted_df[predicted_df['scenario_id'] == scenario_id.split('_')[-1]]
gt_df[gt_df['scenario_id'] == scenario_id.split('_')[-1]]

# display(gt_df)
# display(predicted_df.head())

# 시각화 함수 정의
def plot_trajectories(gt_df, predicted_df, scenario_id):
    fig, ax = plt.subplots(figsize=(10, 6))
    
    # 각 track_id에 대해 GT와 Predicted 데이터를 시각화
    gt_track = gt_df
    pred_track = predicted_df

    # X, Y 축의 최소/최대값을 계산하여 축 범위 설정
    x_min = min(min(gt_track['position_x']), min(pred_track['predicted_trajectory_x'].apply(min)))
    x_max = max(max(gt_track['position_x']), max(pred_track['predicted_trajectory_x'].apply(max)))
    y_min = min(min(gt_track['position_y']), min(pred_track['predicted_trajectory_y'].apply(min)))
    y_max = max(max(gt_track['position_y']), max(pred_track['predicted_trajectory_y'].apply(max)))
    
    ax.set_xlim(x_min, x_max)  # X 축 범위 동적 설정
    ax.set_ylim(y_min, y_max)  # Y 축 범위 동적 설정
    
    ax.set_title(f'GT (Red) and Predicted (Blue) Trajectories for Scenario {scenario_id}')
    ax.set_xlabel('X coordinate')
    ax.set_ylabel('Y coordinate')

    # GT 데이터 시각화 (빨간색)
    ax.plot(gt_track['position_x'], gt_track['position_y'], 'ro-', label=f'GT Track')
    
    lines = []
    for _, row in pred_track.iterrows():
        line, = ax.plot([], [], 'bo-', lw=2, label=f'Predicted Track')  # 예측 데이터는 파란색
        lines.append(line)

    def update(frame):
        for i, line in enumerate(lines):
            x_data = pred_track['predicted_trajectory_x'].iloc[i][:frame]
            y_data = pred_track['predicted_trajectory_y'].iloc[i][:frame]
            line.set_data(x_data, y_data)
        return lines

    anim = FuncAnimation(fig, update, frames=len(pred_track['predicted_trajectory_x'].iloc[0]), interval=500, blit=True)

    # 각 시나리오에 대한 동영상 파일 저장
    anim.save(f'/workspace/infra-guidance/logs/{scenario_id}_trajectory_animation.gif', writer='imagemagick')

    plt.close(fig)  # 애니메이션 후 그래프를 닫기

plot_trajectories(gt_df, predicted_df, scenario_id)


In [ ]:
import pandas as pd

path = '/noah/dataset/infra-guidance/SUMO/sumo_process_step_1/A1/train/raw/1_1000000/scenario_1_1000000.parquet'

# Parquet 파일 읽기
df = pd.read_parquet(path)

# 데이터 확인
print(df['position_y'])